In [2]:
!pip -q install langchain openai tiktoken PyPDF2 faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

# Chat & Query your PDF files

In [3]:
#!pip install langchain_community
!pip install -q sentence-transformers
!pip install -q langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.8 MB/s eta 0:00:00


In [4]:
!pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00


### Basic Chat PDF


In [5]:
from PyPDF2 import PdfReader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

## Reading in the PDF


In [6]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [59]:
from PyPDF2 import PdfReader

# Replace 'impromptu-rh.pdf' with the correct path if needed
pdf_path = '/content/drive/My Drive/Machine Learning with MCMC.pdf'  # Adjust if it's in a subfolder
doc_reader = PdfReader(pdf_path)


In [60]:
doc_reader

In [61]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [62]:
len(raw_text)

230478

In [63]:
raw_text[:100]

'M A C H I N E\nL E A R N I N G\nCheat Sheethttps://github.com/soulmachine/machine-learning-cheat-sheet'

### Text Splitter

This takes the text and splits it into chunks. The chunk size is characters not tokens

In [64]:
# Text splitting
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 500,  # Reduce chunk size to 500
    chunk_overlap  = 100,  # Slightly reduce overlap
    length_function = len,
)
texts = text_splitter.split_text(raw_text)


In [65]:
len(texts)

567

In [66]:
texts[20]

'5.7.2 The false positive vs false\nnegative tradeoff . . . . . . . . . . . . 38\n6 Frequentist statistics . . . . . . . . . . . . . . . . . . . . . . 39\n6.1 Sampling distribution of an estimator . . . 39\n6.1.1 Bootstrap . . . . . . . . . . . . . . . . . . 39\n6.1.2 Large sample theory for the\nMLE * . . . . . . . . . . . . . . . . . . . . 396.2 Frequentist decision theory . . . . . . . . . . . 39\n6.3 Desirable properties of estimators . . . . . . 39'

In [67]:
texts[10]

'2.8.1 Entropy . . . . . . . . . . . . . . . . . . . . 14\n2.8.2 KL divergence . . . . . . . . . . . . . . 14\n2.8.3 Mutual information . . . . . . . . . . 14\n3 Generative models for discrete data . . . . . . . . 17\n3.1 Generative classiﬁer . . . . . . . . . . . . . . . . . 17\n3.2 Bayesian concept learning . . . . . . . . . . . . 17\n3.2.1 Likelihood . . . . . . . . . . . . . . . . . 17\n3.2.2 Prior . . . . . . . . . . . . . . . . . . . . . . 17'

## Making the embeddings

In [16]:
# Use a smaller and faster model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")


<ipython-input-16-992ddb24b03e>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, 

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [68]:
import time

In [69]:
# Start the timer
start_time = time.time()

docsearch = FAISS.from_texts(texts, embedding_model)


# End the timer
end_time = time.time()

# Calculate the time taken
time_taken = end_time - start_time

# Print the time taken in seconds
print(f"Time taken to create FAISS index: {time_taken:.2f} seconds")

Time taken to create FAISS index: 0.84 seconds


In [70]:
docsearch.embedding_function

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='sentence-transformers/paraphrase-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [71]:
query = "What are Bad Markov Chains?"
docs = docsearch.similarity_search(query)

In [72]:
len(docs)

4

In [73]:
docs[0]

Document(metadata={}, page_content='algorithm (global Markov properties)\n1.P contains a chain\np(x;zjy) =p(x;y;z)\np(y)=p(x)p(yjx)p(zjy)\np(y)\n=p(x;y)p(zjy)\np(y)=p(xjy)p(zjy)(10.13)\n2.P contains a fork\np(x;zjy) =p(x;y;z)\np(y)=p(y)p(xjy)p(zjy)\np(y)=p(xjy)p(zjy)\n(10.14)\n3.P contains v-structure\np(x;zjy) =p(x;y;z)\np(y)=p(x)p(z)p(yjx;z)\np(y)̸=p(xjy)p(zjy)\n(10.15)\n10.5.2 Other Markov properties of DGMs\n10.5.3 Markov blanket and full conditionals\nmb(t) =ch(t)[pa(t)[copa (t) (10.16)\n10.5.4 Multinoulli Learning\nMultinoulli Distribution')

**The above similarity Search result we, I will pass to language model**

## Plain QA Chain

In [74]:
!pip install -q langchain_groq

In [26]:
from langchain_groq import ChatGroq
from langchain.chains.question_answering import load_qa_chain


In [27]:
llm = ChatGroq(
    temperature=0,
    groq_api_key = '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',
    model="llama-3.1-70b-versatile"

)



In [28]:
# Pass the `llm` instance to the `load_qa_chain` function
chain = load_qa_chain(llm, chain_type="stuff")
 # we are going to stuff all the docs in at once

<ipython-input-28-f5eec6e358f9>:2: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


In [75]:
for message in chain.llm_chain.prompt.messages:
    print(message)


prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}") additional_kwargs={}
prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}') additional_kwargs={}


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:

In [78]:
query = "What is MCMC?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

'MCMC stands for Markov chain Monte Carlo. It is a method for sampling from high-dimensional distributions. The basic idea behind MCMC is to construct a Markov chain on the state space whose stationary distribution is the target density of interest. This allows for the performance of a random walk on the state space, where the fraction of time spent in each state is proportional to the target density. By drawing samples from the chain, Monte Carlo integration can be performed with respect to the target density.'

In [81]:
query = "what is non parametric classifier?"
query_02 = "has it rained this week?"
docs = docsearch.similarity_search(query_02)
chain.run(input_documents=docs, question=query)

"A non-parametric classifier is a type of classifier that does not make any assumptions about the underlying distribution of the data. In other words, it does not assume a specific form for the probability distribution of the data, such as a normal distribution or a linear relationship.\n\nNon-parametric classifiers are often used when the data does not fit a specific distribution or when the relationship between the features and the target variable is complex and non-linear. They are typically more flexible and robust than parametric classifiers, but can also be more computationally intensive.\n\nExamples of non-parametric classifiers include:\n\n* K-Nearest Neighbors (KNN)\n* Decision Trees\n* Random Forests\n* Support Vector Machines (SVMs) with non-linear kernels\n\nIn contrast, parametric classifiers assume a specific form for the probability distribution of the data and estimate the parameters of that distribution from the data. Examples of parametric classifiers include linear r

In [80]:
query = "what is MAP estimation  and where we use it also tell its benefits and how to implement it?"
docs = docsearch.similarity_search(query,k=4)
chain.run(input_documents=docs, question=query)

'**What is MAP Estimation?**\n\nMAP (Maximum A Posteriori) estimation is a statistical technique used to estimate the parameters of a model by maximizing the posterior probability distribution. It is a Bayesian approach that combines the likelihood of the data with a prior distribution over the parameters to obtain the most likely values of the parameters.\n\n**Where is MAP Estimation Used?**\n\nMAP estimation is widely used in various fields, including:\n\n1. **Machine Learning**: MAP estimation is used in logistic regression, Gaussian mixture models (GMMs), and other probabilistic models to estimate the model parameters.\n2. **Signal Processing**: MAP estimation is used in signal processing to estimate the parameters of a signal model.\n3. **Image Processing**: MAP estimation is used in image processing to estimate the parameters of an image model.\n4. **Natural Language Processing**: MAP estimation is used in natural language processing to estimate the parameters of a language model

In [82]:
chain = load_qa_chain(llm,
                      chain_type="stuff") # we are going to stuff all the docs in at once

In [83]:
query = "what is frequentist statistics how it is different from Bayesian statistics ?"
docs = docsearch.similarity_search(query,k=20)   # Retrieves up to 20 relevant documents
chain.run(input_documents=docs, question=query)  # retrieval-enhanced stuffing

'Frequentist statistics is a branch of statistics that views probability as a long-run frequency of events. In this approach, probabilities represent the proportion of times an event would occur if the experiment were repeated many times. This interpretation of probability is also known as the "frequentist interpretation" or "classical statistics".\n\nThe key characteristics of frequentist statistics are:\n\n1. **Probability as a long-run frequency**: Frequentist statistics views probability as a measure of the proportion of times an event would occur if the experiment were repeated many times.\n2. **Fixed parameters**: In frequentist statistics, the parameters of a model are viewed as fixed, but unknown. The goal is to estimate these parameters from the data.\n3. **Sampling distribution**: Frequentist statistics relies on the concept of a sampling distribution, which is the distribution of a statistic (such as the sample mean) over many repeated samples from the same population.\n4. *

### QA Chain with map-reduce method

In [84]:
chain = load_qa_chain(llm,
                      chain_type="map_rerank",
                      return_intermediate_steps=True # it will return intermmediate steps for each on of these it is calling language model
                      )

query = "What are robust priors?"
docs = docsearch.similarity_search(query,k=10)
results = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
results

<ipython-input-84-1801434fe864>:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'intermediate_steps': [{'answer': 'Helpful Answer: Robust priors are used when we are not very confident in our prior, and we want to make sure it does not have an undue influence on the result. They typically have heavy tails.',
   'score': '80'},
  {'answer': 'Helpful Answer: This document does not answer the question',
   'score': '0'},
  {'answer': 'Helpful Answer: Robust priors are useful but can be computationally expensive to use. They typically have heavy tails, which avoids forcing things to be too close to the prior mean.',
   'score': '80'},
  {'answer': 'Helpful Answer: This document does not answer the question',
   'score': '0'},
  {'answer': 'Helpful Answer: This document does not provide information about what robust priors are, only that they are mentioned in section 5.4.2.',
   'score': '0'},
  {'answer': 'Helpful Answer: This document does not answer the question',
   'score': '0'},
  {'answer': 'Helpful Answer: This document does not answer the question',
   'score

In [85]:
results['output_text'] #THis is final output tex

'Helpful Answer: Robust priors are used when we are not very confident in our prior, and we want to make sure it does not have an undue influence on the result. They typically have heavy tails.'

In [86]:
results['intermediate_steps']

[{'answer': 'Helpful Answer: Robust priors are used when we are not very confident in our prior, and we want to make sure it does not have an undue influence on the result. They typically have heavy tails.',
  'score': '80'},
 {'answer': 'Helpful Answer: This document does not answer the question',
  'score': '0'},
 {'answer': 'Helpful Answer: Robust priors are useful but can be computationally expensive to use. They typically have heavy tails, which avoids forcing things to be too close to the prior mean.',
  'score': '80'},
 {'answer': 'Helpful Answer: This document does not answer the question',
  'score': '0'},
 {'answer': 'Helpful Answer: This document does not provide information about what robust priors are, only that they are mentioned in section 5.4.2.',
  'score': '0'},
 {'answer': 'Helpful Answer: This document does not answer the question',
  'score': '0'},
 {'answer': 'Helpful Answer: This document does not answer the question',
  'score': '0'},
 {'answer': 'Helpful Answer

In [87]:
# check the prompt
chain.llm_chain.prompt.template

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nIn addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:\n\nQuestion: [question here]\nHelpful Answer: [answer here]\nScore: [score between 0 and 100]\n\nHow to determine the score:\n- Higher is a better answer\n- Better responds fully to the asked question, with sufficient level of detail\n- If you do not know the answer based on the context, that should be a score of 0\n- Don't be overconfident!\n\nExample #1\n\nContext:\n---------\nApples are red\n---------\nQuestion: what color are apples?\nHelpful Answer: red\nScore: 100\n\nExample #2\n\nContext:\n---------\nit was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv\n---------\nQuestion: what type was the car?\nHelpful Answer: a sports car or an su

## RetrievalQA Chain
RetrievalQA chain uses load_qa_chain and combines it with the a retriever (in our case the FAISS index)

In [88]:
from langchain.chains import RetrievalQA

# set up FAISS as a generic retriever
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})

# create the chain to answer questions
rqa = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [89]:
rqa("What is Bayesian logistic regression?")

{'query': 'What is Bayesian logistic regression?',
 'result': 'Bayesian logistic regression is a type of logistic regression model that uses Bayesian inference to compute the full posterior over the parameters, p(w|D), where w represents the model parameters and D represents the data. This approach allows for the incorporation of prior knowledge and uncertainty into the model, and can be useful for situations where confidence intervals are needed, such as in contextual bandit problems.',
 'source_documents': [Document(metadata={}, page_content='the results, the amount of data available for any given user\nstarts to look small again (relative to the complexity of the\nproblem).\n7.5 Bayesian linear regression\nTODOChapter 8\nLogistic Regression\n8.1 Representation\nLogistic regression can be binomial or multinomial. The\nbinomial logistic regression model has the following\nform\np(yjx;w) =Ber(yjsigm (wTx)) (8.1)\nwhere w andxare extended vectors, i.e.,\nw= (b;w1;w2;\x01\x01\x01;wD),x= 

In [90]:
query = "What are Generalized linear models ?"
rqa(query)['result']

'Generalized Linear Models (GLMs) are models in which the output density is in the exponential family, and in which the mean parameters are a linear combination of the inputs, passed through a possibly nonlinear function.'

In [91]:
query = "What are Mixture models?"
rqa(query)['result']

'Mixture models are a type of Latent Variable Model (LVM) that represents a discrete latent state. They are used to model complex distributions by combining multiple simpler distributions, known as base distributions. The overall model is a weighted sum of these base distributions, where the weights are the probabilities of each latent state.\n\nIn other words, mixture models are a way of combining multiple probability distributions to create a more complex distribution. This is useful for modeling data that has multiple underlying patterns or structures.\n\nFor example, in a mixture of Gaussians, the base distributions are Gaussian distributions, and the overall model is a weighted sum of these Gaussians. This can be used to model data that has multiple clusters or modes.\n\nMixture models can be used for various applications, including clustering, density estimation, and classification.'